In [ ]:
import pandas as pd

In [ ]:
sweeps = pd.read_csv("analysis/wandb-sweeps.csv")
sweeps

In [ ]:
sweeps['name'].str.contains('accproto').value_counts()

In [ ]:
import ast
is_short =  (sweeps['config_command'].apply(ast.literal_eval).apply(lambda x: x[5]).str.split('/').str[1] == 'short')

sweeps['len'] = is_short.apply(lambda x: 'short' if x else 'long')
sweeps['model'] = sweeps['name'].str.split(':').str[0].str.split('-').str[0]
sweeps['optimization'] = sweeps['name'].str.contains('accproto').apply(lambda x: 'accproto' if x else 'accuracy')
sweeps.groupby(['len', 'model', 'optimization']).count()[['id']]

In [ ]:
runs = pd.read_csv("analysis/wandb-runs.csv")
runs

## Analyze Sweeps

In [ ]:
# 1131 name collisions - most of them from **after** the zlax naming issue was fixed
# Moral - use a better entropy source
counts = runs['best_model'].value_counts()
counts[counts > 1].sum()

## Export

In [ ]:
runsXsweeps = pd.merge(runs, sweeps, left_on='sweep_id', right_on='id', suffixes=('_run', '_sw'))
runsXsweeps

In [ ]:
runsXsweeps_for_eval = runsXsweeps[~runsXsweeps['best_model'].isna()][['model', 'activation_function', 'len', 'backbone', 'name_run', 'sweep_id', 'dataset', 'best_model', 'optimization']].rename(columns={'name_run': 'run_id'})
runsXsweeps_for_eval

In [ ]:
runsXsweeps_for_eval.groupby('run_id').value_counts().sort_values(ascending=False).head(4)

In [ ]:
# three legitimate duplicates
# the filename comes from a randomly seeded value by time - collisions aren't possible,
# especially not while also colliding with the run id
runsXsweeps_for_eval[runsXsweeps_for_eval['run_id'].isin(['2@a1c6ug1d', '1@vse2syxl'])]

In [ ]:
runsXsweeps_for_analysis = runsXsweeps_for_eval.drop_duplicates(['run_id'])
runsXsweeps_for_analysis.shape

In [ ]:
runsXsweeps_for_analysis.to_csv("analysis/wandb-runsXsweeps-foreval.csv", index=False)

In [ ]:
runsXsweeps_accuracy = runsXsweeps_for_analysis[runsXsweeps_for_analysis['optimization'] == 'accuracy']
runsXsweeps_accuracy.groupby(['len', 'optimization', 'model', 'backbone', 'dataset', 'activation_function', 'sweep_id']).count()[['run_id']]

In [ ]:
runsXsweeps_accuracy.to_csv("analysis/wandb-runsXsweeps-foreval-accuracy.csv", index=False)

#### long accuracy
these are analyzed for prototype scores

In [ ]:
runsXsweeps_accuracy_long = runsXsweeps_accuracy[(runsXsweeps_accuracy['len'] == 'long') & (runsXsweeps_accuracy['dataset'] != 'cub200')]
runsXsweeps_accuracy_long.groupby(['model', 'backbone', 'dataset', 'activation_function', 'sweep_id']).count()[['run_id']]

In [ ]:
runsXsweeps_accuracy_long.to_csv("analysis/wandb-runsXsweeps-foreval-accuracy-long.csv", index=False)

# short accuracy

In [ ]:
runsXsweeps_accuracy_short = runsXsweeps_accuracy[runsXsweeps_accuracy['len'] == 'short']
runsXsweeps_accuracy_short.groupby(['model', 'backbone', 'dataset', 'activation_function', 'sweep_id']).count()[['run_id']]

In [ ]:
runsXsweeps_accuracy_short.to_csv("analysis/wandb-runsXsweeps-foreval-accuracy-short.csv", index=False)

### Acc-proto

## 

In [ ]:
runsXsweeps_acc_proto = runsXsweeps_for_analysis[runsXsweeps_for_analysis['optimization'] == 'accproto']
runsXsweeps_acc_proto.groupby(['len', 'optimization', 'model', 'backbone', 'dataset', 'activation_function', 'sweep_id']).count()[['run_id']]

In [ ]:
runsXsweeps_acc_proto.to_csv("analysis/wandb-runsXsweeps-foreval-accproto.csv", index=False)